# 卷积神经网络

在之前的例子中，我们使用了线性层构建了一个简单的模型，它可以拟合数据，这归功于线性层有很多可用于优化的参数。但是这个模型也存在一个问题，即这个模型在记忆训练集方面比在泛化鸟类与飞机的特性方面做的更好。

## 卷积介绍

如果想要识别与物体相应的图案，比如说天空中的一架飞机，我们可能需要看看附近的像素是怎样排列的，但我们对那些彼此相距很远的元素的组合并不关心。

为了将这种直觉转化为数学形式，我们可以计算像素与其相邻元素位置的加权和，而不是图像中所有元素的加权和。这相当于构建权重矩阵，每个输出特征和输出像素位置都有一个权重矩阵，其距离中心象素的一定距离以外的所有权重都为 0 。这仍然是一个加权和，即线性运算。

### 卷积的作用

前面我们提到了一个很重要的特性：我们希望这些局部位置对输出有影响，而不管它们在图中像的位置，也就是说希望神经网络存在平移不变性。以之前的区分鸟和飞机的数据集为例，将这个模型的输入图像作为矢量矩阵的目标，我们需要实现一个相当复杂的权重模式：大多数权重矩阵将被设置为 0 ，因为相对于输入像素对应的项，大多数像素距离输出像素太远而不会产生影响。对于其他权重矩阵，我们必须找一种方法使这些项保持同步，使它们对应输入和输出像素的相同位置，这意味着我们需要将它们初始化为相同值，并且确保在训练期间网络更新时这些像素绑定的权重保持不变。这样，我们就可以确保权重在邻域操作以影响局部模式，并且局部模式无论在哪都可以被识别出来。

当然，这种方法实操性不强，但幸运的是，在图像上有一个现成的、局部的、平移不变的线性操作：卷积。

卷积，更精确的讲：离散卷积，被定义为二维图像的权重矩阵的标量积，即核函数与输入中的每个邻域的标量积。将一个 3\*3 的内核作为一个二维张量：

``` py
weight = torch.tensor([w00, w01, w02],
                      [w10, w11, w12],
                      [w20, w21, w22])
```
以及一个一维的图像通道，M\*N 的图像：
``` py
image = torch.tensor([i00, i01, ... i0N],
                     [i10, i11, ... i1N],
                     ...
                     [iM0, iM1, ... iMN])
```
可以计算出输出图像的一个元素（无偏置）：

``` py
o11 = i11 * w00 + i12 * w01 + i13 * w02 + i21 * w10 + i22 * w11 + i23 * w12 + i31 * w20 + i32 * w21 + i33 * w22
```

![](./data/images/juanji.gif)

也就是说，我们平移输入图像 `i11` 位置上的核函数，然后将每个权重乘以输入图像在相应位置上的值。因此，通过平移所有输入位置上的内核并执行加权和来创建输出图像。对于多通道图像，如 RGB 图像，权重矩阵将是一个 `3*3*3` 的矩阵：每个通道对应一组权重，他们共同作用于输出值。

注意，就像是 `nn.Linear` 权重矩阵中的元素一样。核中的权重是事先不知道的，他们是随机初始化的，并通过反向传播进行更新。还需要注意，相同的核以及核中的权重在整幅图像中被重用，这意味着每个权重的使用都有一个跨越整个图像的历史值，因此关于卷积权值的损失导数来自整个图像的贡献。

现在可以看到它和之前提到的内容的联系了：卷积等价于进行多重线性操作，他们的权重除了个别像素外都为 0 ，并在训练期间接受相同的更新。

综上所述，我们可以得到卷积的以下特征：

* 邻域的局部操作
* 平移不变性
* 模型的参数大幅减少

## 卷积实战

`torch.nn` 模块提供了一维、二维、三维的卷积，其中 `nn.Conv1d` 用于时间序列， `nn.Conv2d` 用于图像， `nn.Conv3d` 用于体数据和视频。

对于 `CIFAR-10` 数据，我们将使用 `nn.Conv2d` ，提供给 `nn.Conv2d` 的参数至少包括输入特征（或通道，因为我们处理的是多通道图像，也就是说，每个像素有多个值）的数量、输入特征的数量以及核的大小等。例如，对于第一个巻积模块，每个像素有 3 个输入特征（RGB 通道），输出特征具有任意数量的通道/特征数。输出通道/特征越多，说明网络的容量越大，我们能借助于这些通道检测到很多不同类型的特征

In [1]:
import torch.nn as nn

conv = nn.Conv2d(3, 16, kernel_size=3)
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

由于卷积核在各个方向上大小相同是很常见的，所以 `PyTorch` 提供了一种快捷方式：每当二维卷积指定 `kernel_size=3` 是为 3\*3